In [252]:
# mlp for regression
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense
from keras.layers import BatchNormalization

In [253]:
# load the dataset
df = pd.read_csv("Memory_SolarTrainingData.csv", index_col=0)

In [254]:
df.tail()
column_mean = []
column_std = []

#df_normalized = df.copy()

# apply normalization techniques
#for column in df_normalized.columns:
#    if column != 'Percent Output':
#        column_mean.append(df_normalized[column].mean())
#        column_std.append(df_normalized[column].std())

#        if df_normalized[column].std() == 0:
#            df_normalized.loc[:,column] = 0
#        else:
#            df_normalized[column] = (df_normalized[column] - df_normalized[column].mean()) / df_normalized[column].std()

# view normalized data


In [4]:
# split into past, current, and future data
pastWeatherAndSupply = df.iloc[:-2].drop(['year', 'day'], axis=1)

currentWeather = df.iloc[-2]
currentWeather = currentWeather.drop(['Percent Output', 'year', 'day']) #dropping supply column for testing
currentWeather = currentWeather.values[:].reshape((1,len(df.iloc[-2])))

nextHourWeather = df.iloc[-1]
nextHourWeather = nextHourWeather.drop(['Percent Output', 'year', 'day']) #dropping supply column for testing
nextHourWeather = nextHourWeather.values[:].reshape((1,len(df.iloc[-1])))

In [256]:
# split into input and output columns
X, y = pastWeatherAndSupply.values[:, :-1], pastWeatherAndSupply.values[:, -1]

In [257]:
# split into train and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(326, 25) (176, 25) (326,) (176,)


In [258]:
# determine the number of input features
n_features = X_train.shape[1]
print(n_features)

25


In [259]:
# define model
model = Sequential()
model.add(BatchNormalization(input_shape=(n_features,)))
model.add(Dense(12, activation='tanh', kernel_initializer='he_normal'))
model.add(Dense(8, activation='selu', kernel_initializer='he_normal'))
model.add(Dense(4, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1, activation='tanh', kernel_initializer='he_normal'))

In [260]:
# compile the model
model.compile(optimizer='adam', loss='mse')

In [261]:
# fit the model
model.fit(X_train, y_train, epochs=80, batch_size=10, verbose=0)

In [262]:
# evaluate the model
error = model.evaluate(X_test, y_test, verbose=0)
print('MSE: %.3f, RMSE: %.3f' % (error, np.sqrt(error)))

MSE: 0.008, RMSE: 0.087


In [263]:
# Making a prediction for integration with control code
# IMPROVEMENT AREA: Sometimes the model will predict a number greater than 1. This should not happen.
current_prediction = model.predict(currentWeather)
if (current_prediction < 0):
    current_prediction = 0
print('Predicted: %.3f' % current_prediction)
print('Real: %.3f' % df.iloc[-2][-1])
print(abs(df.iloc[-2][-1]-current_prediction)/df.iloc[-2][-1])

future_prediction = model.predict(nextHourWeather)
if (future_prediction < 0):
    future_prediction = 0
print('Predicted: %.3f' % future_prediction)
print('Real: %.3f' % df.iloc[-1][-1])
print(abs(df.iloc[-1][-1]-future_prediction)/df.iloc[-1][-1])

1/1 [==============================] - 0s 177ms/step
Predicted: 0.605
Real: 0.621
[[0.02560828]]
1/1 [==============================] - 0s 70ms/step
Predicted: 0.472
Real: 0.416
[[0.13434428]]


In [264]:
# Return predictions in a list
#return([current_prediction, future_prediction])